In [5]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from math import pi
#from Shape_Visualisation import draw_shape
%matplotlib inline

In [6]:
df=pd.read_csv("urop.csv")


# extract healthy echo data
df = df.drop(["IDshort",'SERNO','HistoCode'], 1)
mask=df["Cardiac_Category (0=good; 1=poor)"]==0
healthy_df=df[mask]
echo=healthy_df.iloc[:,19:55]

# fill in missing data
for col in echo.columns:
    echo[col].fillna(echo[col].mean(),inplace=True)

    
# data that form part of cardiac geometry    
left_ventricle=echo.iloc[:,1:8]
aorta=echo.iloc[:,8:9]
left_atrium=echo.iloc[:,9:10]
sinus=echo.iloc[:,-4:-2]

In [44]:
x_m=np.array(left_ventricle.mean())

rows=np.array([])
for i in range(len(left_ventricle)):
    x=left_ventricle.iloc[i:i+1,:]
    rows=np.append(rows,x)
rows_2d=rows.reshape(42,7)
rows_2d[0]

array([0.7, 0.9, 4. , 2.7, 0.6, 1.1, 2. ])

In [105]:
class Vector():
    def __init__(self,a):
        septum_d,septum_s=a[0],a[1]
        dim_d,dim_s=a[2],a[3]
        wall_d,wall_s=a[4],a[5]
        # Estimate shape using  parametric eqn of ellipse
        # x = u + a cos(t) ; y = v + b sin(t)

        u=0     #x-position of the center
        v=0   #y-position of the center
        a=4 #radius on the x-axis
        b=dim_d/2   #radius on the y-axis
        indices=[]

        t_all= np.linspace(0, 2*pi, 50)
        x_all=(u+a*np.cos(t_all))[::-1]
        y_all=(v+b*np.sin(t_all))[::-1]

        #plt.plot(x_all,y_all)


        #remove x,y coordinates where y< (dim_s/2)

        for i,(x,y) in enumerate(zip(x_all,y_all)):
            if y<=dim_s/2:
                indices+=[i]


        x=np.delete(x_all,indices)
        y=np.delete(y_all,indices)

        y_s=y + septum_d
        y_w= -y - wall_d

        x_more=np.linspace(x[-1],6,10)
        y_more=np.linspace(y[-1],y[-1],10)

        # Split the ellipse into 2 (left and right)
        self.x_left=x[:np.argmax(y)+1]
        self.y_left=y[:np.argmax(y)+1]

        ys_left=y_s[:np.argmax(y_s)+1]
        xs_left=x[:np.argmax(y_s)+1]
        yw_left=y_w[:np.argmin(y_w)+1]
        xw_left=xs_left

        self.x_right=x[np.argmax(y_s):]
        self.y_right=y[np.argmax(y_s):]
        
    def get_vectors(self):
        
        return self.x_left, self.y_left, self.x_right, self.y_right

    def get_length(self):
        return len(self.x_left), len(self.x_right)
        

In [106]:
class Vectorise:
    
    def __init__(self,rows_2d):
        self.list_a , self.list_b = [], []
        self.rows_2d = rows_2d
        self.XY_full = np.array([])
        for row in rows_2d:
            a=Vector(row)
            A,B=a.get_length()
            self.list_a.append(A)
            self.list_b.append(B)
            
    
    def full_vectors(self):
        a_min= min(self.list_a)
        b_min= min(self.list_b)
        
        for row in self.rows_2d:
            a=Vector(row)
            x_left,y_left,x_right,y_right=a.get_vectors()
            
            idx_l,idx_r=[],[]
            if len(x_left)>min(list_a):
                idx_l+=range(min(list_a),len(x_left))

            if len(x_right)>min(list_b):
                idx_r+=range(min(list_b),len(x_right))

            x_left = np.delete(x_left,idx_l,axis=None)
            y_left = np.delete(y_left,idx_l,axis=None)

            x_right= np.delete(x_right,idx_r,axis=None)
            y_right= np.delete(y_right,idx_r,axis=None)


            XY_left =np.array([])
            for i in range(len(x_left)):
                xy=np.concatenate(([x_left[i]],[y_left[i]]),axis=None)
                XY_left=np.append(XY_left,xy)


            XY_right =np.array([])
            for i in range(len(x_left)):
                xy=np.concatenate(([x_right[i]],[y_right[i]]),axis=None)
                XY_right=np.append(XY_right,xy)


            XY=np.concatenate((XY_left,XY_right),axis=None)
            
            x_more=np.linspace(XY[-2],6,10)
            y_more=np.linspace(XY[-1],XY[-1],10)
            
            XY_more =np.array([])
            
            for i in range(len(x_more)):
                xy=np.concatenate(([x_more[i]],[y_more[i]]),axis=None)
                XY_more=np.append(XY_more,xy)


            XY=np.concatenate((XY,XY_more))
            self.XY_full= np.concatenate((self.XY_full,XY))
        
        # Note: need to include as variables in the future
        self.XY_full=self.XY_full.reshape(42,48)
        
        return self.XY_full
    
    def dataframe(self):
        df=pd.DataFrame(self.XY_full)
        
        return df

In [107]:
a=Vectorise(rows_2d)
a.full_vectors()
df=a.dataframe()
df

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,-2.689204,1.480556,-2.288467,1.640345,-1.850153,1.773199,-1.381460,1.876937,-0.890084,1.949856,...,4.610399,1.391365,4.957799,1.391365,5.305199,1.391365,5.6526,1.391365,6.0,1.391365
1,-2.689204,2.257848,-2.288467,2.501525,-1.850153,2.704128,-1.381460,2.862329,-0.890084,2.973530,...,4.610399,2.121832,4.957799,2.121832,5.305199,2.121832,5.6526,2.121832,6.0,2.121832
2,-3.045784,1.393691,-2.689204,1.591598,-2.288467,1.763370,-1.850153,1.906189,-1.381460,2.017707,...,4.610399,1.495717,4.957799,1.495717,5.305199,1.495717,5.6526,1.495717,6.0,1.495717
3,-3.352352,1.172900,-3.045784,1.393691,-2.689204,1.591598,-2.288467,1.763370,-1.850153,1.906189,...,4.610399,1.495717,4.957799,1.495717,5.305199,1.495717,5.6526,1.495717,6.0,1.495717
4,-3.045784,1.620571,-2.689204,1.850695,-2.288467,2.050431,-1.850153,2.216498,-1.381460,2.346171,...,4.610399,1.739206,4.957799,1.739206,5.305199,1.739206,5.6526,1.739206,6.0,1.739206
5,-3.045784,1.166811,-2.689204,1.332500,-2.288467,1.476310,-1.850153,1.595879,-1.381460,1.689243,...,4.610399,1.252229,4.957799,1.252229,5.305199,1.252229,5.6526,1.252229,6.0,1.252229
6,-3.045784,1.588160,-2.689204,1.813681,-2.288467,2.009422,-1.850153,2.172168,-1.381460,2.299248,...,4.610399,1.704422,4.957799,1.704422,5.305199,1.704422,5.6526,1.704422,6.0,1.704422
7,-3.045784,1.264045,-2.689204,1.443542,-2.288467,1.599336,-1.850153,1.728869,-1.381460,1.830013,...,4.610399,1.356581,4.957799,1.356581,5.305199,1.356581,5.6526,1.356581,6.0,1.356581
8,-3.045784,1.166811,-2.689204,1.332500,-2.288467,1.476310,-1.850153,1.595879,-1.381460,1.689243,...,4.610399,1.252229,4.957799,1.252229,5.305199,1.252229,5.6526,1.252229,6.0,1.252229
9,-3.045784,1.361280,-2.689204,1.554584,-2.288467,1.722362,-1.850153,1.861859,-1.381460,1.970784,...,4.610399,1.460933,4.957799,1.460933,5.305199,1.460933,5.6526,1.460933,6.0,1.460933


In [108]:
def all_vectors(a):


    septum_d,septum_s=a[0],a[1]
    dim_d,dim_s=a[2],a[3]
    wall_d,wall_s=a[4],a[5]


    # Estimate shape using  parametric eqn of ellipse
    # x = u + a cos(t) ; y = v + b sin(t)

    u=0     #x-position of the center
    v=0   #y-position of the center
    a=4 #radius on the x-axis
    b=dim_d/2   #radius on the y-axis
    indices=[]

    t_all= np.linspace(0, 2*pi, 50)
    x_all=(u+a*np.cos(t_all))[::-1]
    y_all=(v+b*np.sin(t_all))[::-1]
    
    #plt.plot(x_all,y_all)

    
    #remove x,y coordinates where y< (dim_s/2)

    for i,(x,y) in enumerate(zip(x_all,y_all)):
        if y<=dim_s/2:
            indices+=[i]


    x=np.delete(x_all,indices)
    y=np.delete(y_all,indices)
    
    y_s=y + septum_d
    y_w= -y - wall_d
    
    x_more=np.linspace(x[-1],6,10)
    y_more=np.linspace(y[-1],y[-1],10)
    
    # Split the ellipse into 2 (left and right)
    x_left=x[:np.argmax(y)+1]
    y_left=y[:np.argmax(y)+1]
    
    ys_left=y_s[:np.argmax(y_s)+1]
    xs_left=x[:np.argmax(y_s)+1]
    yw_left=y_w[:np.argmin(y_w)+1]
    xw_left=xs_left

    x_right=x[np.argmax(y_s):]
    y_right=y[np.argmax(y_s):]
    
    #return x_left,y_left,x_right,y_right
    def get_vectors():
        

SyntaxError: unexpected EOF while parsing (<ipython-input-108-2e50f2f8f3ec>, line 55)

NameError: name 'x_left' is not defined